# `qp` Demo

In this notebook we use the `qp` module to approximate some standard 1-D PDFs using sets of quantiles, and assess the accuracy of the quantile parametrization(TM).

### Requirements

To run `qp`, you will need to first install the module. 

In [ ]:
import numpy as np
import scipy.stats as sps
import scipy.interpolate as spi

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

import pdf

## The `qp.PDF` Class

This is the basic element of `qp` - an object representing a probability density function. This class is stored in the module `pdf.py`, which you can read here if you want.

In [ ]:
# This now exists in a separate module.  Classes will not appear in the notebook!

# import numpy as np
# import matplotlib.pyplot as plt

# class PDF(object):

#     def __init__(self, truth=None):
#         self.truth = truth

#     def evaluate(self, loc):#PDF

#         return

#     def integrate(self, limits):#CDF

#         return

#     def quantize(self, num_points):#inverse CDF (PPF)

#         quanta = 1./num_points
#         points = np.linspace(0.+quanta, 1.-quanta, num_points)
#         print(points)
#         self.quantiles = self.truth.ppf(points)
#         print(self.quantiles)

#         return self.quantiles

#     def interpolate(self):

#         return

#     def plot(self, limits, num_points):

#         x = np.linspace(limits[0], limits[1], 100)

#         y = [0., 1.]
#         plt.plot(x, self.truth.pdf(x), color='k', linestyle='-', lw=1., alpha=1., label='true pdf')
#         plt.vlines(self.quantize(num_points), y[0], y[1], color='k', linestyle='--', lw=1., alpha=1., label='quantiles')
#         plt.legend()
#         plt.xlabel('x')
#         plt.ylabel('probability density')
#         plt.savefig('plot.png')

#         return# !cat pdf.py

## Approximating a Gaussian

Let's summon a PDF object, and initialize it with a standard functiom - a Gaussian.

In [ ]:
dist = sps.norm(loc=0, scale=1)
P = pdf.PDF(truth=dist)

Now, let's compute a set of evenly spaced quantiles. These will be carried by the `PDF` object as `p.quantiles`.

In [ ]:
quantiles = P.quantize(percent=10)

Let's visualize the PDF object in order to compare the truth and the approximation.

In [ ]:
bounds = (-5.0, 5.0)
npoints = 9
P.plot(limits=bounds)

The dashed lines show the 10 percentiles that we asked for. 10% of the total probability mass is enclosed between each pair. Note that the quantiles refer to the probability distribution *between the bounds*, because we are not able to integrate numerically over an infinite range.  